In [185]:
import pandas as pd
import nltk
import ast
import sys
from pathlib import Path
from nltk.metrics import jaccard_distance
from nltk.util import ngrams
from typing import List, Set
from scipy.stats import pearsonr
import numpy as np

# dir_nltk_data = 'c:\\Users\\maric\\Documents\\MASTERS\\PrimerSemestre\\IHLT\\FinalProject\\IHLT\\Lib\\site-packages\\nltk\\tag'

# Download required resource
nltk.download('averaged_perceptron_tagger_eng')# , download_dir=dir_nltk_data)

# Add the project directory to the Python path
project_dir = Path.cwd().parent
sys.path.append(str(project_dir))

from Preprocessing.preprocessingUtils import TextPreprocessor

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\maric\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [186]:
# Load the training dataset
train_df = pd.read_csv('../Preprocessing/STS_train.csv')

# Display the DataFrame
train_df.head()

,0,1,gs
0,"['But', 'other', 'sources', 'close', 'to', 'th...","['But', 'other', 'sources', 'close', 'to', 'th...",4.00
1,"['Micron', 'has', 'declared', 'its', 'first', ...","['Micron', ""'s"", 'numbers', 'also', 'marked', ...",3.75
2,"['The', 'fines', 'are', 'part', 'of', 'failed'...","['Perry', 'said', 'he', 'backs', 'the', 'Senat...",2.80
3,"['The', 'American', 'Anglican', 'Council', ','...","['The', 'American', 'Anglican', 'Council', ','...",3.40
4,"['The', 'tech-loaded', 'Nasdaq', 'composite', ...","['The', 'technology-laced', 'Nasdaq', 'Composi...",2.40


In [187]:
# Turn the 2 first columns from strings to actual lists of strings
train_df.iloc[:, :2] = train_df.iloc[:, :2].map(ast.literal_eval)

train_df.head()

,0,1,gs
0,"[But, other, sources, close, to, the, sale, sa...","[But, other, sources, close, to, the, sale, sa...",4.00
1,"[Micron, has, declared, its, first, quarterly,...","[Micron, 's, numbers, also, marked, the, first...",3.75
2,"[The, fines, are, part, of, failed, Republican...","[Perry, said, he, backs, the, Senate, 's, effo...",2.80
3,"[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...",3.40
4,"[The, tech-loaded, Nasdaq, composite, rose, 20...","[The, technology-laced, Nasdaq, Composite, Ind...",2.40


In [188]:
# Create the TextPreprocessor
preprocessor = TextPreprocessor()

# Remove punctuation
train_df = preprocessor.remove_punctuation(train_df)
train_df = preprocessor.remove_empty_strings(train_df)

# POS-tagging the words

n=len(train_df)
train_df_POS = pd.DataFrame(columns=['0','1'])

function_words_tag = {'IN', 'CC', 'DT', 'PDT', 'WDT'}

for i in range(n):
    train_df_POS.loc[i,'0'] = nltk.pos_tag(train_df.loc[i,'0']) 
    train_df_POS.loc[i,'1'] = nltk.pos_tag(train_df.loc[i,'1']) 

train_df_POS.head()


,0,1
0,"[(But, CC), (other, JJ), (sources, NNS), (clos...","[(But, CC), (other, JJ), (sources, NNS), (clos..."
1,"[(Micron, NNP), (has, VBZ), (declared, VBN), (...","[(Micron, NNP), (s, NN), (numbers, NNS), (also..."
2,"[(The, DT), (fines, NNS), (are, VBP), (part, N...","[(Perry, NNP), (said, VBD), (he, PRP), (backs,..."
3,"[(The, DT), (American, JJ), (Anglican, NNP), (...","[(The, DT), (American, JJ), (Anglican, NNP), (..."
4,"[(The, DT), (tech-loaded, JJ), (Nasdaq, NNP), ...","[(The, DT), (technology-laced, JJ), (Nasdaq, N..."


In [189]:
# the function words (prepositions, conjunctions, articles) carry less semantics than content words and thus removing them might eliminate
# the noise and provide a more accurate estimate of semantic similarity.

function_words_tag = {'IN', 'CC', 'DT', 'PDT', 'WDT'}

for i in range(n):
    for tag in function_words_tag:
        j=0
        while j < (len(train_df_POS.loc[i,'0'])):
            if train_df_POS.loc[i,'0'][j][1]==tag:
                train_df_POS.loc[i,'0'].pop(j)
            j=j+1
        j=0
        while j < (len(train_df_POS.loc[i,'1'])):
            if train_df_POS.loc[i,'1'][j][1]==tag:
                train_df_POS.loc[i,'1'].pop(j)
            j=j+1
            
train_df_POS.head()

,0,1
0,"[(other, JJ), (sources, NNS), (close, RB), (to...","[(other, JJ), (sources, NNS), (close, RB), (to..."
1,"[(Micron, NNP), (has, VBZ), (declared, VBN), (...","[(Micron, NNP), (s, NN), (numbers, NNS), (also..."
2,"[(fines, NNS), (are, VBP), (part, NN), (failed...","[(Perry, NNP), (said, VBD), (he, PRP), (backs,..."
3,"[(American, JJ), (Anglican, NNP), (Council, NN...","[(American, JJ), (Anglican, NNP), (Council, NN..."
4,"[(tech-loaded, JJ), (Nasdaq, NNP), (composite,...","[(technology-laced, JJ), (Nasdaq, NNP), (Compo..."


In [205]:
train_df_1 = pd.DataFrame(columns=['0','1','gs'])

for i in range(n):
    sentence=[]
    for j in range(len(train_df_POS.loc[i,'0'])):
        sentence.append(train_df_POS.loc[i,'0'][j][0])
    train_df_1.loc[i,'0']=sentence
    sentence=[]
    for k in range(len(train_df_POS.loc[i,'1'])):
        sentence.append(train_df_POS.loc[i,'1'][k][0])
    train_df_1.loc[i,'1']=sentence

train_df_1['gs'] = train_df['gs']
train_df_1.head(10)

,0,1,gs
0,"[other, sources, close, to, sale, said, Vivend...","[other, sources, close, to, sale, said, Vivend...",4.000
1,"[Micron, has, declared, its, first, quarterly,...","[Micron, s, numbers, also, marked, first, quar...",3.750
2,"[fines, are, part, failed, Republican, efforts...","[Perry, said, he, backs, Senate, s, efforts, i...",2.800
3,"[American, Anglican, Council, represents, Epis...","[American, Anglican, Council, represents, Epis...",3.400
4,"[tech-loaded, Nasdaq, composite, rose, 20.96, ...","[technology-laced, Nasdaq, Composite, Index, I...",2.400
5,"[Amgen, shares, gained, 93, cents, 1.45, perce...","[Shares, Allergan, were, up, 14, cents, 78.40,...",1.333
6,"[U.S, prosecutors, have, arrested, more, 130, ...","[More, 130, people, have, been, arrested, 17, ...",4.600
7,"[Chavez, said, investigators, feel, confident,...","[Albuquerque, Mayor, Martin, Chavez, said, inv...",3.800
8,"[Authorities, said, scientist, properly, quara...","[scientist, also, quarantined, himself, home, ...",4.200
9,"[support, will, come, free, software, upgrade,...","[upgrade, will, be, available, free, download,...",2.600


In [212]:
def n_gram_overlap(tokens1: List[str], tokens2: List[str], n: int) -> float:
    """
    Computes the n-gram overlap between two tokenized sentences.

    Parameters:
        tokens1 (List[str]): Tokenized first sentence as a list of strings.
        tokens2 (List[str]): Tokenized second sentence as a list of strings.
        n (int): The size of n-grams.

    Returns:
        float: The n-gram overlap ratio.
    """
    def generate_ngrams(tokens: List[str], n: int) -> Set[str]:
        """
        Generates n-grams for a given list of tokens.

        Parameters:
            tokens (List[str]): The input tokens.
            n (int): The size of n-grams.

        Returns:
            Set[str]: A set of n-grams.
        """
        return set([' '.join(tokens[i:i+n]) for i in range(len(tokens) - n + 1)])

    # Generate n-grams for both token lists
    ngrams_s1 = generate_ngrams(tokens1, n)
    ngrams_s2 = generate_ngrams(tokens2, n)

    # Compute the intersection 
    intersection = ngrams_s1.intersection(ngrams_s2)

    # Compute the n gram overlap when posible
    if len(intersection)==0:
        ngo=0
    else:
        ngo=2/((len(ngrams_s1)+len(ngrams_s2))/len(intersection))

    return float(ngo)


In [213]:
syntactic_features=pd.DataFrame(columns=['POS_tagging_unigrams','POS_tagging_bigrams','POS_tagging_trigrams'])

In [219]:
for i in range(n):
    # unigrams
    syntactic_features.loc[i,'POS_tagging_unigrams'] = n_gram_overlap(train_df_1.loc[i,'0'],train_df_1.loc[i,'1'],1)
    # bigrams
    syntactic_features.loc[i,'POS_tagging_bigrams'] = n_gram_overlap(train_df_1.loc[i,'0'],train_df_1.loc[i,'1'],2)
    # trigrams
    syntactic_features.loc[i,'POS_tagging_trigrams'] = n_gram_overlap(train_df_1.loc[i,'0'],train_df_1.loc[i,'1'],3)


# Convert all columns in a DataFrame to numeric, coercing errors into NaN.
syntactic_features['POS_tagging_unigrams'] = pd.to_numeric(syntactic_features['POS_tagging_unigrams'], errors='coerce') 
syntactic_features['POS_tagging_bigrams'] = pd.to_numeric(syntactic_features['POS_tagging_bigrams'], errors='coerce') 
syntactic_features['POS_tagging_trigrams'] = pd.to_numeric(syntactic_features['POS_tagging_trigrams'], errors='coerce') 

syntactic_features.head()
    

,POS_tagging_unigrams,POS_tagging_bigrams,POS_tagging_trigrams
0,0.702703,0.594595,0.514286
1,0.571429,0.421053,0.352941
2,0.500000,0.250000,0.090909
3,0.756757,0.742857,0.727273
4,0.230769,0.000000,0.000000


In [223]:
pearsonr(syntactic_features['POS_tagging_unigrams'], train_df['gs'])[0]

np.float64(0.5113723483644254)